# Test Document Similarity v2


In [35]:
import json

# load Data from JSON
with open('articles_20230422-1203.json', 'r', encoding='utf8') as file:
    data_json = json.load(file)

In [36]:
# Filtern nach Attribut "source" mit Wert "Watson"
data_watson = [d for d in data_json if d.get('source') == 'Watson']

print(f'len start data {len(data_watson)}')

len start data 77220


In [37]:
# just take 100

data_watson_1000 = []
counter = 0
for d in data_watson:
    if counter < 1000:
        counter = counter + 1
        data_watson_1000.append(d)
    else:
        break

print(f'len start data {len(data_watson_1000)}')

len start data 1000


In [38]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Dokumente aus der Sammlung abrufen und in eine Liste speichern
docs = data_watson_1000

# TfidfVectorizer-Objekt initialisieren und auf den Text anwenden
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([doc['text'] for doc in docs])
print(vectors.shape)

(1000, 44693)


In [39]:
from sklearn.metrics.pairwise import cosine_similarity

duplicates = set()

# Schwellenwert für die Ähnlichkeit festlegen
threshold = 0.90

# Cosine-Similaritätsmatrix berechnen
for i,v in enumerate(vectors):
    cosine_sim = cosine_similarity(vectors, v)
    # print(cosine_sim)
    # break
    for j in range(len(cosine_sim)):
        if cosine_sim[j][0] > threshold and j != i:
            duplicates.add(j)
            # print(cosine_sim[j][0])
            if cosine_sim[j][0] > 0.98:
                 continue
            print(docs[j]['text'])
            print(docs[i]['text'])
            print(cosine_sim[j][0])

print(len(duplicates))

# Eindeutige Dokumente auswählen
unique_docs_watson = []
for i in range(len(docs)):
    if i not in duplicates:
        unique_docs_watson.append(docs[i])

print(len(unique_docs_watson))

 Erfreulich verlief trotz der zwei schwierigen Jahre mit Corona auch die sportliche Entwicklung. Die Mitgliederzahlen bei den angeschlossenen Klubs und Tenniscentern stiegen im Vergleich zum letzten Jahr um 6,7 Prozent, die Zahl der Lizenzen von Wettkampfspielern um 2,3 Prozent. Sie sind damit auf dem höchsten Stand seit 2016. (ram/sda)Tennis zeigt sich sehr zufrieden mit dem zu Ende gehenden Jahr 2022. Der schweizerische Tennisverband schliesst das Geschäftsjahr 2021/22 mit einem Gewinn von gut 236'000 Franken ab und kann zusätzlich 800'000 in den Fonds für Davis Cup, Billie Jean Cup und Spitzensport überweisen.ubs waren nicht im Spiel) waren ungefähr gleich.at Pius Suter mit den Detroit Red Wings eine 1:4-Niederlage gegen die Minnesota Wild erlitten. Suter gab einen Schuss aufs Tor ab und stand während 14:33 Minuten auf dem Eis. Bei Minnesota wurde Frederick Gaudreau zum Matchwinner, der Kanadier schoss das erste und das letzte Tor der Partie. (ram) Die Atlanta Hawks treten im Kampf 

In [40]:
import pandas as pd

df = pd.DataFrame(unique_docs_watson)

records = df.to_dict('records')

# Datensätze in eine JSON-Datei schreiben
with open('clean-data_watson_1000_v2.json', 'w', encoding='utf8') as file:
    json.dump(records, file)

In [42]:
import pymongo
import json

# Verbindung zur MongoDB-Datenbank herstellen
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["mydatabase"]

# JSON-Datei lesen
with open('clean-data_watson_1000_v2.json', 'r') as file:
    data = json.load(file)

# Datensätze in die MongoDB-Datenbank schreiben
collection = db["mycollection"]
collection.insert_many(data)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

docs_2 = list(map(lambda p: p.get('text'), data_watson_1000))

# TfidfVectorizer-Objekt initialisieren und auf den Text anwenden
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(docs_2)

# Cosine-Similaritätsmatrix berechnen
cosine_sim = cosine_similarity(vectors)

# print(cosine_sim[0,0])

# Duplikate finden und entfernen
duplicates = []
for i in range(len(cosine_sim)):
    for j in range(i+1, len(cosine_sim)):
        if cosine_sim[i,j] > 0.8:  # Schwellenwert für Ähnlichkeit anpassen
            duplicates.append(j)


print(f'len of new col: {len(duplicates)}')

len of new col: 396


In [ ]:

# Duplikate finden und entfernen
duplicates = set()
for i in range(len(cosine_sim)):
    for j in range(i+1, len(cosine_sim)):
        if cosine_sim[i,j] > 0.7:  # Schwellenwert für Ähnlichkeit anpassen
            duplicates.add(j)

# Eindeutige Dokumente auswählen
unique_docs = []
for i in range(len(docs)):
    if i not in duplicates:
        unique_docs.append(docs[i])

print(f'len of new unique collection: {len(unique_docs)}')